In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("~/Downloads/toy_data.csv")

In [3]:
# df.dropna(axis = 1, thresh= 0.9*df.shape[0], inplace= True)
# nunique = df.apply(pd.Series.nunique)
# cols_to_drop = nunique[nunique == 1].index
# df.drop(cols_to_drop, axis = 1, inplace= True)

In [4]:
#required_cols = ['agency_name','agency_abbr','agency_code','state_name','state_abbr','state_code','loan_amount_000s', 'applicant_sex_name', 'applicant_sex', 'tract_to_msamd_income', 'population','census_tract_number','number_of_1_to_4_family_units','minority_population', 'applicant_income_000s']

In [5]:
required_cols = ['applicant_sex', 'applicant_sex_name','agency_abbr','agency_name','applicant_income_000s','number_of_1_to_4_family_units','population']

In [6]:
df = df[required_cols]

In [7]:
nunique = df.apply(pd.Series.nunique)
discrete_columns = nunique[nunique < 100].index

In [8]:
numeric_columns = df.select_dtypes(include=['number']).columns
object_columns = df.select_dtypes(include=['object']).columns
df[numeric_columns] = df[numeric_columns].fillna(0)
df[object_columns] = df[object_columns].fillna("NA")

df.isna().any()

applicant_sex                    False
applicant_sex_name               False
agency_abbr                      False
agency_name                      False
applicant_income_000s            False
number_of_1_to_4_family_units    False
population                       False
dtype: bool

In [9]:
df.shape

(999999, 7)

In [10]:
from importlib import reload
import synthesizer,transformer,sampler,conditional,models
# reload(models)
# reload(sampler)
# reload(synthesizer)
# reload(transformer)
# reload(conditional)
data = df#.loc[0:100000,:]
model = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model.fit(data, discrete_columns, 100,log_frequency= False)


Condtional Cross Entropy Loss tensor(1.1699, grad_fn=<DivBackward0>)
Epoch 1, Loss G: 1.1897, Loss D: -0.1993, Loss Conditional D: -0.6226
Condtional Cross Entropy Loss tensor(1.7620, grad_fn=<DivBackward0>)
Epoch 2, Loss G: 2.1324, Loss D: -0.2967, Loss Conditional D: -0.9594
Condtional Cross Entropy Loss tensor(1.7091, grad_fn=<DivBackward0>)
Epoch 3, Loss G: 2.2900, Loss D: -0.3437, Loss Conditional D: -1.1121
Condtional Cross Entropy Loss tensor(0.7715, grad_fn=<DivBackward0>)
Epoch 4, Loss G: 1.4706, Loss D: -0.3654, Loss Conditional D: -1.2118
Condtional Cross Entropy Loss tensor(1.5258, grad_fn=<DivBackward0>)
Epoch 5, Loss G: 2.2116, Loss D: -0.4239, Loss Conditional D: -1.4190
Condtional Cross Entropy Loss tensor(1.4971, grad_fn=<DivBackward0>)
Epoch 6, Loss G: 2.4340, Loss D: -0.4111, Loss Conditional D: -1.4250
Condtional Cross Entropy Loss tensor(0.6794, grad_fn=<DivBackward0>)
Epoch 7, Loss G: 1.4559, Loss D: -0.3361, Loss Conditional D: -1.0914
Condtional Cross Entropy Lo

In [51]:
sampled = model.sample(1000000)

tensor(0.0249, grad_fn=<DivBackward0>)
tensor(0.0309, grad_fn=<DivBackward0>)
tensor(0.0238, grad_fn=<DivBackward0>)
tensor(0.0336, grad_fn=<DivBackward0>)
tensor(0.0431, grad_fn=<DivBackward0>)
tensor(0.0402, grad_fn=<DivBackward0>)
tensor(0.0278, grad_fn=<DivBackward0>)
tensor(0.0329, grad_fn=<DivBackward0>)
tensor(0.0318, grad_fn=<DivBackward0>)
tensor(0.0294, grad_fn=<DivBackward0>)
tensor(0.0324, grad_fn=<DivBackward0>)
tensor(0.0243, grad_fn=<DivBackward0>)
tensor(0.0149, grad_fn=<DivBackward0>)
tensor(0.0266, grad_fn=<DivBackward0>)
tensor(0.0361, grad_fn=<DivBackward0>)
tensor(0.0387, grad_fn=<DivBackward0>)
tensor(0.0376, grad_fn=<DivBackward0>)
tensor(0.0337, grad_fn=<DivBackward0>)
tensor(0.0245, grad_fn=<DivBackward0>)
tensor(0.0238, grad_fn=<DivBackward0>)
tensor(0.0504, grad_fn=<DivBackward0>)
tensor(0.0226, grad_fn=<DivBackward0>)
tensor(0.0233, grad_fn=<DivBackward0>)
tensor(0.0350, grad_fn=<DivBackward0>)
tensor(0.0245, grad_fn=<DivBackward0>)
tensor(0.0325, grad_fn=<D

In [67]:
sampled = model.sample(1000000,col_index= 8)

4.9727, grad_fn=<DivBackward0>)
tensor(5.2420, grad_fn=<DivBackward0>)
tensor(5.1189, grad_fn=<DivBackward0>)
tensor(5.1838, grad_fn=<DivBackward0>)
tensor(5.1186, grad_fn=<DivBackward0>)
tensor(5.0160, grad_fn=<DivBackward0>)
tensor(5.2930, grad_fn=<DivBackward0>)
tensor(4.9576, grad_fn=<DivBackward0>)
tensor(5.3611, grad_fn=<DivBackward0>)
tensor(5.1743, grad_fn=<DivBackward0>)
tensor(5.2243, grad_fn=<DivBackward0>)
tensor(5.1097, grad_fn=<DivBackward0>)
tensor(4.9212, grad_fn=<DivBackward0>)
tensor(4.9741, grad_fn=<DivBackward0>)
tensor(4.9665, grad_fn=<DivBackward0>)
tensor(5.2874, grad_fn=<DivBackward0>)
tensor(5.3389, grad_fn=<DivBackward0>)
tensor(5.0646, grad_fn=<DivBackward0>)
tensor(5.0252, grad_fn=<DivBackward0>)
tensor(5.0029, grad_fn=<DivBackward0>)
tensor(5.0064, grad_fn=<DivBackward0>)
tensor(5.1603, grad_fn=<DivBackward0>)
tensor(5.0165, grad_fn=<DivBackward0>)
tensor(5.1554, grad_fn=<DivBackward0>)
tensor(5.0331, grad_fn=<DivBackward0>)
tensor(5.1032, grad_fn=<DivBackw

In [22]:
agency_abbr = np.array(["CFPB", "HUD", "FDIC", "FRS", "NCUA", "OCC"])
applicant_sex = np.array([2, 3, 1,4])

In [58]:
og = (pd.crosstab(data.agency_abbr, data.applicant_sex).reindex(applicant_sex, axis="columns")/data.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,6.30,1.87,14.33,0.02,22.52
FDIC,2.44,0.64,5.95,0.02,9.05
FRS,0.90,0.21,2.10,0.00,3.21
HUD,15.23,4.92,32.54,0.02,52.71
NCUA,2.84,0.77,4.94,0.00,8.56
OCC,1.08,0.22,2.65,0.01,3.95
Total,28.78,8.64,62.51,0.07,100.00


In [61]:
new = (pd.crosstab(sampled.agency_abbr, sampled.applicant_sex).reindex(applicant_sex, axis="columns")/sampled.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,0.08,0.00,28.17,0.01,28.26
FDIC,0.00,0.00,0.32,0.00,0.32
FRS,0.00,0.00,0.27,0.00,0.27
HUD,0.00,0.00,61.71,0.00,61.72
NCUA,0.00,0.05,8.95,0.00,9.00
OCC,0.00,0.00,0.44,0.00,0.44
Total,0.09,0.05,99.85,0.01,100.00


In [62]:
og = (pd.crosstab(data.applicant_sex_name, data.applicant_sex).reindex(applicant_sex, axis="columns")/data.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,28.78,0.00,0.00,0.00,28.78
Not provided,0.00,8.64,0.00,0.00,8.64
Male,0.00,0.00,62.51,0.00,62.51
Not applicable,0.00,0.00,0.00,0.07,0.07
Total,28.78,8.64,62.51,0.07,100.00


In [63]:
new = (pd.crosstab(sampled.applicant_sex_name, sampled.applicant_sex).reindex(applicant_sex, axis="columns")/sampled.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,0.00,0.00,0.33,0.00,0.33
Not provided,0.00,0.00,0.45,0.00,0.45
Male,0.08,0.05,99.01,0.01,99.15
Not applicable,0.00,0.00,0.07,0.00,0.07
Total,0.09,0.05,99.85,0.01,100.00


In [104]:
new - og

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,-22.48,0.17,6.47,0.75,-15.10
Not provided,0.17,-13.57,2.19,0.36,-10.85
Male,1.47,0.60,8.16,0.60,10.84
Not applicable,1.65,1.22,9.93,2.30,15.10
Total,-19.19,-11.57,26.74,4.01,0.00


In [64]:
og = (pd.crosstab(data.applicant_sex_name,data.agency_abbr).reindex(agency_abbr, axis="columns")/data.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,6.30,15.23,2.44,0.90,2.84,1.08,28.78
Not provided,1.87,4.92,0.64,0.21,0.77,0.22,8.64
Male,14.33,32.54,5.95,2.10,4.94,2.65,62.51
Not applicable,0.02,0.02,0.02,0.00,0.00,0.01,0.07
Total,22.52,52.71,9.05,3.21,8.56,3.95,100.00


In [65]:
new = (pd.crosstab(sampled.applicant_sex_name,sampled.agency_abbr).reindex(agency_abbr, axis="columns")/sampled.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,0.33,0.00,0.00,0.00,0.00,0.00,0.33
Not provided,0.02,0.00,0.00,0.00,0.42,0.01,0.45
Male,27.87,61.71,0.32,0.26,8.56,0.43,99.15
Not applicable,0.04,0.00,0.00,0.00,0.02,0.00,0.07
Total,28.26,61.72,0.32,0.27,9.00,0.44,100.00


In [218]:
new - og

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,0.09,-6.62,-1.18,3.38,4.32,0.02,0.0
Not provided,2.15,-2.74,-0.53,0.41,0.98,-0.27,0.0
Male,-0.44,-1.84,5.30,-1.19,-2.88,1.07,0.0
Not applicable,0.00,0.00,0.00,0.00,0.00,0.01,0.0
Total,1.79,-11.20,3.58,2.59,2.42,0.82,0.0


In [164]:
og = (pd.crosstab(data.agency_abbr,data.applicant_sex).reindex(applicant_sex, axis="columns")/data.shape[0] * 100)
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,6.00,3.30,13.38,0.01,22.70
FDIC,2.09,0.97,4.96,0.01,8.04
FRS,0.79,0.32,1.82,0.00,2.94
HUD,14.84,8.36,31.63,0.01,54.84
NCUA,2.36,1.26,4.08,0.01,7.70
OCC,0.97,0.41,2.39,0.00,3.78
Total,27.06,14.62,58.28,0.05,100.00


In [155]:
new = (pd.crosstab(sampled.agency_abbr,sampled.applicant_sex).reindex(applicant_sex, axis="columns")/sampled.shape[0] * 100)
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,6.25,5.68,12.43,0.04,24.40
FDIC,0.92,0.47,10.00,0.06,11.45
FRS,4.29,0.78,0.56,0.01,5.65
HUD,8.68,5.74,29.12,0.06,43.59
NCUA,6.86,2.24,1.24,0.03,10.38
OCC,0.95,0.16,3.42,0.01,4.54
Total,27.95,15.07,56.77,0.20,100.00


In [104]:
new - og

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,-2.65,-2.17,-0.82,0.09,-5.57
FDIC,0.68,-0.55,-0.22,0.03,-0.07
FRS,0.09,0.02,1.47,0.03,1.60
HUD,-0.76,-0.70,0.38,0.16,-0.92
NCUA,-0.86,-0.46,3.52,0.04,2.25
OCC,0.41,0.41,1.83,0.06,2.69
Total,-3.10,-3.44,6.14,0.40,0.00


In [11]:
#import synthesizer,transformer,sampler,conditional,models,data
torch.save(model, "/Users/sahanalva/Counterfactual Research/big.pt")

In [218]:
from importlib import reload
import synthesizer,transformer,sampler,conditional,models
reload(models)
reload(sampler)
reload(synthesizer)
reload(transformer)
reload(conditional)
data = df.loc[0:100000,:]
model1 = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model1.fit(data, discrete_columns, 30,log_frequency= False)
model2 = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model2.fit(data, discrete_columns, 30,log_frequency= True)

AttributeError: 'list' object has no attribute 'shape'

In [156]:
sampled = pd.concat([model1.sample(50000),model2.sample(50000)], axis = 0)

In [176]:
original_applicant_sex_p = (data.applicant_sex_name.value_counts()/data.shape[0]).to_numpy()
applicant_sex_values = (data.applicant_sex_name.value_counts()/data.shape[0]).index.to_numpy()
original_agency_abbr_p = (data.agency_abbr.value_counts()/data.shape[0]).to_numpy()
agency_abbr_values = (data.agency_abbr.value_counts()/data.shape[0]).index.to_numpy()


In [211]:
sampled_data_list = []
sampled_all = model2.sample(500000)
for name,value in zip(applicant_sex_values, original_applicant_sex_p):
    temp = sampled_all[sampled_all['applicant_sex_name'] == name]
    sampled_data_list.append(temp.sample(n= int(value*100000)))



In [212]:
sampled = pd.concat(sampled_data_list,axis = 0 )

In [203]:
sampled_data_list = []
sampled_all = model2.sample(500000)
for name,value in zip(agency_abbr_values, original_agency_abbr_p):
    temp = sampled_all[sampled_all['agency_abbr'] == name]
    sampled_data_list.append(temp.sample(n= int(value*100000)))


In [68]:
new = (pd.crosstab(sampled.agency_abbr, sampled.agency_name)/sampled.shape[0] * 100)
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

agency_name,Consumer Financial Protection Bureau,Department of Housing and Urban Development,Federal Deposit Insurance Corporation,Federal Reserve System,National Credit Union Administration,Office of the Comptroller of the Currency,Total
agency_abbr,,,,,,,
CFPB,93.04,1.34,0.57,0.56,1.99,0.71,98.22
FDIC,0.13,0.00,0.00,0.00,0.01,0.00,0.15
FRS,0.09,0.00,0.00,0.00,0.01,0.00,0.11
HUD,0.48,0.02,0.00,0.01,0.00,0.01,0.52
NCUA,0.50,0.00,0.01,0.01,0.31,0.02,0.85
OCC,0.14,0.00,0.00,0.00,0.01,0.00,0.15
Total,94.39,1.38,0.59,0.58,2.33,0.74,100.00


In [17]:
del1 = np.array([[-3.6]])

In [18]:
x0 = np.array([[1],[-0.9]])

In [19]:
x0*np.transpose(del1)

array([[-3.6 ],
       [ 3.24]])

In [8]:
x0

array([[-0.44,  0.88],
       [-0.88,  1.75]])

In [20]:
k = [1,2,3]

In [28]:
k[-2:]

[2, 3]

In [199]:
p = np.array([[0.6, 0.1,0.05,0.25], [0.8, 0.1,0.1,0]])

In [200]:
idx = np.random.choice(2, 10) 

In [201]:
p

array([[0.6 , 0.1 , 0.05, 0.25],
       [0.8 , 0.1 , 0.1 , 0.  ]])

In [202]:
a = p[idx]

In [203]:
r = np.expand_dims(np.random.rand(a.shape[0]), axis=1)

In [204]:
r

array([[0.24429936],
       [0.60124273],
       [0.49534919],
       [0.63003839],
       [0.63278085],
       [0.77043964],
       [0.04708423],
       [0.14637997],
       [0.30450196],
       [0.70424133]])

In [205]:
(a.cumsum(axis=1) > r)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [206]:
(a.cumsum(axis=1) > r).argmax(axis=1)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [210]:
idx

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1])

In [157]:
interval = np.array([[0,4],[4,3]])

In [158]:
interval[idx,0]

array([0, 4, 4, 0, 0, 4, 4, 0, 4, 0])

In [161]:
r

array([[0.92816249],
       [0.74734661],
       [0.67106556],
       [0.23983039],
       [0.42317289],
       [0.77550191],
       [0.61703935],
       [0.28192594],
       [0.21640551],
       [0.7812334 ]])

In [214]:
a

array([[0.8 , 0.1 , 0.1 , 0.  ],
       [0.8 , 0.1 , 0.1 , 0.  ],
       [0.8 , 0.1 , 0.1 , 0.  ],
       [0.8 , 0.1 , 0.1 , 0.  ],
       [0.8 , 0.1 , 0.1 , 0.  ],
       [0.6 , 0.1 , 0.05, 0.25],
       [0.6 , 0.1 , 0.05, 0.25],
       [0.6 , 0.1 , 0.05, 0.25],
       [0.8 , 0.1 , 0.1 , 0.  ],
       [0.8 , 0.1 , 0.1 , 0.  ]])

In [201]:
perm = np.arange(10)

In [202]:
perm

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [209]:
idx[np.random.shuffle(perm)]

array([[1, 1, 1, 1, 1, 0, 0, 0, 1, 1]])

In [212]:
idx[perm]

array([0, 1, 0, 1, 1, 0, 1, 1, 1, 1])

In [221]:
np.random.choice(np.array([1,2,6,4,5,7,90,2]))

7

In [120]:
from torch.nn import functional
import torch
functional.cross_entropy(
                    data,
                    torch.argmax(c, dim=1),
                    reduction='none'
                )

tensor([3.3141e-01, 9.4083e+00, 2.4757e-03])

In [121]:
data = torch.tensor([[-11.0,-2.0,-3,8,1],[-3.0,4,3.0,2,-5], [-11, -12,-13,8,2]]).float()

In [122]:
data

tensor([[-11.,  -2.,  -3.,   8.,   1.],
        [ -3.,   4.,   3.,   2.,  -5.],
        [-11., -12., -13.,   8.,   2.]])

In [115]:
c = torch.tensor([[0,0,0,1,0],[0,0,0,0,1],[0,0,0,1,0]]).float()

In [123]:
torch.argmax(c, dim=1)

tensor([3, 4, 3])

In [176]:
import numpy as np

In [208]:
np.random.randint(low=0, high=100) * np.ones_like(o)

array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
       44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44])

In [194]:
o = np.random.choice(np.arange(10), 30)  

In [195]:
o

array([3, 7, 3, 6, 5, 4, 5, 6, 1, 0, 6, 0, 0, 4, 6, 1, 8, 7, 7, 0, 9, 8,
       5, 1, 1, 4, 7, 3, 8, 6])

In [207]:
np.count_nonzero(np.array([1,2,3,1,3,1,3,0,0,0,0]))

7

In [236]:
sampled

,applicant_sex,applicant_sex_name
0,2,Female
1,2,Female
2,2,Female
3,2,Female
4,2,Female
...,...,...
99995,2,Female
99996,2,Female
99997,2,Female
99998,2,Female


In [47]:
reload(sns)

<module 'seaborn' from '/anaconda2/envs/python36/lib/python3.7/site-packages/seaborn/__init__.py'>

In [42]:
sampled.head()

,applicant_sex,applicant_sex_name,agency_abbr,agency_name,applicant_income_000s,number_of_1_to_4_family_units,population
0,2,Female,FDIC,Federal Deposit Insurance Corporation,77.792309,1436.687763,3072.177619
1,1,Male,FDIC,Federal Deposit Insurance Corporation,214.910052,1178.864820,3332.980128
2,2,Female,FDIC,Federal Deposit Insurance Corporation,170.884616,410.201332,3382.068227
3,3,"Information not provided by applicant in mail,...",FDIC,Federal Deposit Insurance Corporation,330.547750,2568.157828,6060.632136
4,1,Male,FDIC,Federal Deposit Insurance Corporation,110.474369,2291.985421,6645.118951


In [48]:
sns.displot(data, x="applicant_income_000s", hue="applicant_sex_name", multiple="stack")


AttributeError: module 'seaborn' has no attribute 'displot'

In [39]:
seaborn.kdeplot

<function seaborn.distributions.kdeplot(data, data2=None, shade=False, vertical=False, kernel='gau', bw='scott', gridsize=100, cut=3, clip=None, legend=True, cumulative=False, shade_lowest=True, cbar=False, cbar_ax=None, cbar_kws=None, ax=None, **kwargs)>

In [52]:
sampled.to_csv("/Users/sahanalva/Counterfactual Research/sampled_big.csv")

In [50]:
data.to_csv("/Users/sahanalva/Counterfactual Research/data_big.csv")